In [1]:
import numpy as np
import scipy as sp
from sklearn import tree
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [2]:
train_data = pd.read_csv("train_data.csv")
dev_data = pd.read_csv("dev_data.csv")
test_data = pd.read_csv("test_data.csv")

In [3]:
#visualizing the dataset
train_data

,Sample Name,Sample Type,3-Carene,Camphene,Caryophyllene Oxide,Eucalyptol,Geraniol,Guaiol,Isopulegol,Linalool,...,CBD-A,CBD,CBDV,CBDV-A,delta-9 CBG-A,delta-9 CBG,CBC,Aromas,Flavors,Effects
0,Chem Valley Kush,"Flower, Inhalable",0.000,0.008,0.013,0.001,0.000,0.031,0.009,0.086,...,0.08,0.00,0.00,0.00,0.44,0.06,0.00,"['Chemical', 'Earthy', 'Lemon', 'Pine', 'Woody']","['Chemical', 'Diesel', 'Lemon', 'Peppery', 'Pi...","['Creative', 'Euphoria', 'Happy', 'Uplifting']"
1,Blueberry Pie,"Hybrid, Flower, Inhalable",0.000,0.014,0.003,0.000,0.002,0.000,0.000,0.250,...,0.02,0.00,0.00,0.00,0.64,0.09,0.00,"['Berry', 'Earthy', 'Nutty', 'Sweet', 'Vanilla']","['Berry', 'Blueberry', 'Sweet', 'Vanilla']","['Cerebral', 'Euphoria', 'Happy', 'Relaxing', ..."
2,Gods Gift,"Flower, Inhalable",0.000,0.003,0.009,0.000,0.000,0.000,0.000,0.030,...,0.05,0.00,0.00,0.00,0.42,0.03,0.01,"['Earthy', 'Grape', 'Spicy', 'Sweet']","['Berry', 'Citrus', 'Grape', 'Sweet', 'Woody']","['Euphoria', 'Happy', 'Relaxing', 'Sleepy']"
3,Blood Diamond OG - The Humboldt Cure,"Concentrate, Inhalable",0.094,0.015,0.085,0.020,0.000,0.385,0.000,0.161,...,0.22,0.00,0.00,0.00,3.57,0.38,0.33,"['Earthy', 'Lemon', 'Pungent', 'Sweet']","['Citrus', 'Herbal', 'Pine', 'Woody']","['Body High', 'Cerebral', 'Long Lasting', 'Rel..."
4,Green Dragon (Live Resin),"Concentrate, Inhalable",0.000,0.000,0.081,0.000,0.000,0.000,0.000,0.230,...,0.19,0.00,0.03,0.00,1.33,0.70,0.18,"['Earthy', 'Mint', 'Pungent', 'Sweet']","['Candy', 'Mint', 'Pine', 'Skunky', 'Sweet', '...","['Cerebral', 'Euphoria', 'Focus', 'Happy', 'Re..."
5,BLACKWATER,"Flower, Inhalable",0.000,0.004,0.000,0.000,0.000,0.042,0.000,0.015,...,0.05,0.00,0.00,0.00,0.16,0.02,0.05,"['Earthy', 'Grape', 'Lemon', 'Pine', 'Sweet']","['Berry', 'Grape', 'Lemon', 'Pine', 'Sweet']","['Euphoria', 'Happy', 'Hungry', 'Sleepy']"
6,Green Door Kush,"Flower, Inhalable",0.000,0.004,0.008,0.000,0.000,0.000,0.000,0.069,...,0.06,0.00,0.00,0.00,1.72,0.12,0.01,"['Citrus', 'Earthy', 'Herbal', 'Lemon']","['Citrus', 'Fruity', 'Lemon', 'Sweet']","['Creative', 'Energizing', 'Euphoria', 'Happy'..."
7,Jack the Ripper,"Concentrate, Inhalable",0.006,0.000,0.236,0.017,0.000,0.384,0.014,0.081,...,0.22,0.00,0.00,0.00,3.55,0.46,0.15,"['Citrus', 'Lemon', 'Pungent', 'Sour']","['Lemon', 'Tangy']","['Creative', 'Energizing', 'Euphoria', 'Happy']"
8,Now N Later (NUG),"Concentrate, Inhalable",0.000,0.007,0.034,0.000,0.000,0.000,0.000,0.267,...,0.17,0.00,0.00,0.00,1.89,0.47,0.41,"['Citrus', 'Earthy', 'Herbal', 'Sour', 'Spicy'...","['Citrus', 'Pine', 'Sour', 'Spicy', 'Woody']","['Body High', 'Cerebral', 'Euphoria', 'Happy',..."
9,Blue Dream Haze,"Flower, Inhalable",0.000,0.008,0.016,0.005,0.000,0.002,0.016,0.144,...,0.10,0.00,0.00,0.00,0.58,0.06,0.00,"['Blueberry', 'Earthy', 'Herbal', 'Sweet']","['Berry', 'Honey', 'Sweet']","['Energizing', 'Happy', 'Relaxing', 'Uplifting']"


In [4]:
### checking for missing data
print(train_data.isnull().sum())
print(dev_data.isnull().sum())
print(test_data.isnull().sum())

Sample Name              0
Sample Type              0
3-Carene                 0
Camphene                 0
Caryophyllene Oxide      0
Eucalyptol               0
Geraniol                 0
Guaiol                   0
Isopulegol               0
Linalool                 0
Ocimene                  0
Terpinolene              0
alpha-Bisabolol          0
alpha-Humulene           0
alpha-Pinene             0
alpha-Terpinene          0
beta-Caryophyllene       0
beta-Myrcene             0
beta-Ocimene           306
beta-Pinene              0
delta-Limonene           0
gamma-Terpinene          0
p-Cymene                 0
delta-9 THC-A          306
delta-9 THC              0
delta-8 THC              1
THC-A                    1
THCV                     2
CBN                      0
CBD-A                    1
CBD                      0
CBDV                     0
CBDV-A                   1
delta-9 CBG-A            0
delta-9 CBG              0
CBC                      4
Aromas                   0
F

In [5]:
#deleting blank columns
train_data.dropna(axis = 'columns', how = 'all', inplace = True)
dev_data.dropna(axis = 'columns', how = 'all', inplace = True)
test_data.dropna(axis = 'columns', how = 'all', inplace = True) 
#deleting rows containing a blank feature
train_data.dropna(inplace = True)
dev_data.dropna(inplace = True)
test_data.dropna(inplace = True) 

In [6]:
#splitting the datasets into x and y vectors
dev_classes = dev_data.iloc[:, -3] #just aroma
dev_features = dev_data.iloc[:, 2:-3] #droping string data values at the beginning of the list

train_classes = train_data.iloc[:, -3] #just aroma
train_features = train_data.iloc[:, 2:-3] #droping string data values at the beginning of the list

test_classes = test_data.iloc[:, -3] #just aroma
test_features = test_data.iloc[:, 2:-3] #droping string data values at the beginning of the list

In [7]:
#used to turn aromas from a string to a list
def get_array_from_string(entry):
    complete_list = []
    entry = entry.replace("[", "")
    entry = entry.replace("]", "")
    entry = entry.replace("'", "")
    entry = entry.replace(" ", "")
    entry_array = entry.split(",")

    for aroma in entry_array:
        complete_list.append(aroma)

    return complete_list

#accepts a dataframe of string terpene classifications
#returns an array of containing 10 binary categorical aroma values
def terpine_one_hot_encoder_simpler(df):
    output = np.zeros((len(df.index), 10))
    i = 0
    for element in df:
        arr = get_array_from_string(element)
        for entry in arr:
            if entry == "Chemical" or entry == "Fuel" or entry == "Diesel":
                output[i][0] = 1
            if entry == "Earthy":
                output[i][1] = 1
            if entry == "Woody" or entry == "Pine":
                output[i][2] = 1
            if entry == "Citrus" or entry == "Lemon" or entry == "Lime" or entry == "Orange" or entry == "Citrus":
                output[i][3] = 1
            if entry == "Fruity" or entry == "Pineapple" or entry == "Strawberry" or entry == "Cherry" or entry == "Apple" or entry == "Grapefruit" or entry == "Mango" or entry == "Blueberry" or entry == "Grape" or entry == "Berry" or entry == "Tropical":
                output[i][4] = 1
            if entry == "Skunky" or entry == "Grassy" or entry == "Musky" or entry == "Dank" or entry == "Pungent" or entry == "Cheese" or entry == "Fragrant" or entry == "Kush" or entry == "Harsh":
                output[i][5] = 1
            if entry == "Nutty":
                output[i][6] = 1
            if entry == "Sweet" or entry == "Vanilla" or entry == "BubbleGum" or entry == "Candy" or entry == "Creamy" or entry == "Caramel" or entry == "Chocolate" or entry == "Mint":
                output[i][7] = 1
            if entry == "Spicy" or entry == "Pepper" or entry == "Hash":
                output[i][8] = 1
            if entry == "Herbal" or entry == "Floral" or entry == "Coffee" or entry == "Flowery" or entry == "Sage":
                output[i][9] = 1
        i+=1
    return output

def accuracy(predictions, actual):
    incorrect = 0
    incorrect_list = []
    partially_correct = 0
    partially_correct_list = []
    totally_correct = 0
    totally_correct_list = []
    correct = 0
    total_correct = 0
    total_aromas = 0
    for i in range(0,len(predictions)):
        for j in range(0,len(predictions[0])):
            #print("predictions[i][j]: " + str(predictions[i][j]))
            #print("actual[i][j]: " + str(actual[i][j]))
            if predictions[i][j] == actual[i][j] == 1:
                correct += 1
                total_correct += 1
                #print("correct: " + str(correct))
        if correct == np.count_nonzero(actual[i] == 1):
            totally_correct += 1
            totally_correct_list.append(i)
        elif correct > 0:
            partially_correct += 1
            partially_correct_list.append(i)
        else:
            incorrect += 1
            incorrect_list.append(i)
        #print("total correct elements: " + str(correct))
        #print("total labels: " + str(np.count_nonzero(actual[i] == 1)))
        total_aromas += np.count_nonzero(actual[i] == 1)
        correct = 0
    print("incorrect: " + str(incorrect))
    print("partially_correct: " + str(partially_correct))
    print("totally_correct: " + str(totally_correct))
    print(f"Of {total_aromas} aromas {total_correct} were correct for an accuracy of {total_correct/total_aromas}")
    #print(totally_correct_list)
    return total_correct/total_aromas

In [9]:
#simpler-one-hot-encoding aroma classes
train_classes_ohe2 = terpine_one_hot_encoder_simpler(train_classes)
dev_classes_ohe2 = terpine_one_hot_encoder_simpler(dev_classes)
test_classes_ohe2 = terpine_one_hot_encoder_simpler(test_classes)

In [10]:
#grid search for simplified aromas using depth, minimum samples to split, and minimum samples per leaf
best_accuracy = 0
best_max_depth = 0
best_max_features = 0
for i in range(1, 25):
    for j in range(1, 30):
        clf = tree.DecisionTreeClassifier(max_depth = i, max_features = j, random_state = 0)
        clf = clf.fit(train_features, train_classes_ohe2)
        predictions = clf.predict(dev_features)
        if (accuracy(predictions, dev_classes_ohe2)) > best_accuracy:
            best_accuracy = accuracy(predictions, dev_classes_ohe2)
            best_max_depth = i
            best_max_features = j
print(f"Best max depth: {best_max_depth}")
print(f"Best max features: {best_max_features}")
print(f"Best accuracy: {best_accuracy}")

incorrect: 8
partially_correct: 66
totally_correct: 1
Of 277 aromas 114 were correct for an accuracy of 0.41155234657039713
incorrect: 8
partially_correct: 66
totally_correct: 1
Of 277 aromas 114 were correct for an accuracy of 0.41155234657039713
incorrect: 7
partially_correct: 66
totally_correct: 2
Of 277 aromas 130 were correct for an accuracy of 0.4693140794223827
incorrect: 7
partially_correct: 66
totally_correct: 2
Of 277 aromas 130 were correct for an accuracy of 0.4693140794223827
incorrect: 7
partially_correct: 66
totally_correct: 2
Of 277 aromas 130 were correct for an accuracy of 0.4693140794223827
incorrect: 7
partially_correct: 66
totally_correct: 2
Of 277 aromas 130 were correct for an accuracy of 0.4693140794223827
incorrect: 7
partially_correct: 66
totally_correct: 2
Of 277 aromas 130 were correct for an accuracy of 0.4693140794223827
incorrect: 7
partially_correct: 66
totally_correct: 2
Of 277 aromas 130 were correct for an accuracy of 0.4693140794223827
incorrect: 7
p

In [11]:
clf = tree.DecisionTreeClassifier(max_depth = 17, max_features = 22, random_state = 0)
clf = clf.fit(train_features, train_classes_ohe2)
predictions = clf.predict(dev_features)
accuracy(predictions, dev_classes_ohe2)

incorrect: 6
partially_correct: 65
totally_correct: 4
Of 277 aromas 156 were correct for an accuracy of 0.5631768953068592


0.5631768953068592

In [12]:
#evaluating on test data
clf2 = tree.DecisionTreeClassifier(max_depth = 17, max_features = 22, random_state = 0)
clf2 = clf2.fit(train_features, train_classes_ohe2)
predictions2 = clf2.predict(test_features)
accuracy(predictions2, test_classes_ohe2)

incorrect: 12
partially_correct: 69
totally_correct: 10
Of 314 aromas 139 were correct for an accuracy of 0.4426751592356688


0.4426751592356688